In [ ]:
!git clone https://github.com/thibui0908/Twitter-Bot-Detection.git

Cloning into 'Twitter-Bot-Detection'...
remote: Enumerating objects: 5423, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 5423 (delta 3), reused 7 (delta 2), pack-reused 5412
Receiving objects: 100% (5423/5423), 1.66 GiB | 27.21 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Checking out files: 100% (2677/2677), done.


In [ ]:
## Vertex & Graph class

class Vertex:
    def __init__(self, key):
        self.id = key
        self.connectedTo = {}

    def addNeighbor(self, neighbor, weight = 0):
        self.connectedTo[neighbor] = weight
    
    def __str__(self):
        return str(self.id) + ' connectedTo: ' + str([x.id for x in self.connectedTo])
    
    def getConnections(self):
        return self.connectedTo.keys()
    
    def getId(self):
        return self.id
    
    def getWeight(self, neighbor):
        return self.connectedTo[neighbor]
    
class Graph:
    def __init__(self):
        self.vertexList = {}
        self.numOfVertices = 0
    
    def addVertex(self, key):
        self.numOfVertices = self.numOfVertices + 1
        newVertex = Vertex(key)
        self.vertexList[key] = newVertex
        return newVertex

    def getVertex(self, n):
        if n in self.vertexList:
            return self.vertexList[n]
        else:
            return None
    
    def __contains__(self, n):
        return n in self.vertexList
    
    def addEdge(self, f, t, cost = 0):
        if f not in self.vertexList:
            nv = self.addVertex(f)
        if t not in self.vertexList:
            nv = self.addVertex(t)
        self.vertexList[f].addNeighbor(self.vertexList[t], cost)
    
    def getVertices(self):
        return self.vertexList.keys()
    
    def __iter__(self):
        return iter(self.vertexList.values())

In [ ]:
import pandas as pd

In [ ]:
!ls

sample_data  Twitter-Bot-Detection


In [ ]:
nodes = pd.read_csv('Twitter-Bot-Detection/data/bot_user_nodes/bot_user_nodes_176.csv', names = ['Id'],header=None)
nodes = nodes['Id']
num_nodes = nodes.shape[0]
print(num_nodes)
nodes

568


0                593661392
1       891357264086528002
2      1265307045160538116
3                883085263
4      1274567819225632769
              ...         
563    1240687632675409929
564     811576970303705088
565    1251905803839307777
566              343717073
567              233657084
Name: Id, Length: 568, dtype: int64

In [ ]:
edges = pd.read_csv('Twitter-Bot-Detection/data/bot_user_edge/bot_user_edge_176.csv', names = ['Source', 'Target'],header=None)
edges.shape
edges

,Source,Target
0,1255066195235143681,1576229254085345281
1,1255066195235143681,3029183129
2,1255066195235143681,1309466216860397568
3,1255066195235143681,540018030
4,1255066195235143681,783315967
...,...,...
7909,1255066195235143681,2611447940
7910,1255066195235143681,1279183267313393665
7911,1255066195235143681,1251905803839307777
7912,1255066195235143681,2445671278


In [ ]:
g = Graph()

for i in range(num_nodes):
  g.addVertex(nodes[i])

for i in range(len(edges)):
  edge = edges.iloc[i]
  g.addEdge(edge['Source'], edge['Target'])
  g.addEdge(edge['Target'], edge['Source'])

for v in g:
  for w in v.getConnections():
    print(" %s-%s" % (v.getId(), w.getId()))

 593661392-1255066195235143681
 891357264086528002-1255066195235143681
 1265307045160538116-1255066195235143681
 883085263-1255066195235143681
 1274567819225632769-1255066195235143681
 1290085662511837185-1255066195235143681
 1285575131906138113-1255066195235143681
 1294905591387680768-1255066195235143681
 1054333491410800640-1255066195235143681
 1273015990733897735-1255066195235143681
 1170914522879512577-1255066195235143681
 1267442046291279872-1255066195235143681
 1251566172627222536-1255066195235143681
 1296014795687092224-1255066195235143681
 1267078237655633920-1255066195235143681
 1228663880202760192-1255066195235143681
 1176476897514942464-1255066195235143681
 1260462833197887490-1255066195235143681
 1256400086881505281-1255066195235143681
 1182576210016178176-1255066195235143681
 1294786235546963969-1255066195235143681
 1098713131914874880-1255066195235143681
 1295639404065816576-1255066195235143681
 137875162-1255066195235143681
 1292829783232000001-1255066195235143681
 12866

In [ ]:
from graphviz import Digraph

dot = Digraph(comment='connection-graph', format='png', strict = True)

for i in range(num_nodes):
  dot.node(str(nodes[i]))

edges

,Source,Target
0,1255066195235143681,1576229254085345281
1,1255066195235143681,3029183129
2,1255066195235143681,1309466216860397568
3,1255066195235143681,540018030
4,1255066195235143681,783315967
...,...,...
7909,1255066195235143681,2611447940
7910,1255066195235143681,1279183267313393665
7911,1255066195235143681,1251905803839307777
7912,1255066195235143681,2445671278


In [ ]:
for i in range(len(edges)):
  edge = edges.iloc[i]
  dot.edge(str(edge['Source']), str(edge['Target']))

In [ ]:
import os
import csv
import re

In [ ]:
## Graph classification

graph_classification = {}
isBot = True
bot = 1
real = 2
node_labels = {}
count = 0
graph_number = 1
graph_edges = {}

graph_classification = {}

graph_classification[bot] = []
graph_classification[real] = []
directories = ['Twitter-Bot-Detection/data/bot_user_edge/',
               'Twitter-Bot-Detection/data/real_user_edge/']

bot_list = []
real_list = []

def build_graph_edges(edge_list, count):
  for edge in edge_list:
    source = int(edge[0])
    target = int(edge[1])
    if source not in node_labels:
      node_labels[source] = count
      count += 1
    if node_labels[source] not in graph_edges:
      graph_edges[node_labels[source]] = set()
    else:
      if target not in node_labels:
        node_labels[target] = count
        count += 1
      graph_edges[node_labels[source]].add(node_labels[target])
  return count

FILE_LIMIT = 500

for directory in directories:
  isBot = True if directory == 'Twitter-Bot-Detection/data/bot_user_edge/' else False
  print('directory: ', directory, 'bot: ', isBot)
  count_file = 0
  for file in [file for file in os.listdir(directory) if file.endswith('.csv')]:
    if count_file == FILE_LIMIT:
      break
    with open(directory + file, 'r') as csv_file:
      edge_reader = csv.reader(csv_file)
      edge_list = list(edge_reader)
      graph_edges = {}
      count = build_graph_edges(edge_list, count)
      if isBot:
        graph_classification[bot].append([graph_number, graph_edges])
      else:
        graph_classification[real].append([graph_number, graph_edges])
      graph_number += 1
    count_file += 1
    if count_file%20==0: 
      print("Reached file #", count_file)

directory:  Twitter-Bot-Detection/data/bot_user_edge/ bot:  True
Reached file # 20
Reached file # 40
Reached file # 60
Reached file # 80
Reached file # 100
Reached file # 120
Reached file # 140
Reached file # 160
Reached file # 180
Reached file # 200
Reached file # 220
Reached file # 240
Reached file # 260
Reached file # 280
Reached file # 300
Reached file # 320
Reached file # 340
Reached file # 360
Reached file # 380
Reached file # 400
Reached file # 420
Reached file # 440
Reached file # 460
Reached file # 480
Reached file # 500
directory:  Twitter-Bot-Detection/data/real_user_edge/ bot:  False
Reached file # 20
Reached file # 40
Reached file # 60
Reached file # 80
Reached file # 100
Reached file # 120
Reached file # 140
Reached file # 160
Reached file # 180
Reached file # 200
Reached file # 220
Reached file # 240
Reached file # 260
Reached file # 280
Reached file # 300
Reached file # 320
Reached file # 340
Reached file # 360
Reached file # 380
Reached file # 400
Reached file # 420
Re

In [ ]:
print(graph_number)
print(count)
print(len(node_labels))

1001
20438856
20438856


In [ ]:
import pickle as pk
import numpy as np

In [ ]:
with open('saved_dictionary.pkl', 'wb') as f:
    pk.dump(graph_classification, f)

In [ ]:
with open('saved_dictionary.pkl', 'rb') as f:
    graph_classification = pk.load(f)

In [ ]:
b = []
c = []
nodes = {}

for classification in graph_classification.keys():
  for graph_list in graph_classification[classification]:
    a = []
    graph_edges = graph_list[1]
    edges = set()
    n = {}
    labeled_edges = {}
    for source in graph_edges.keys():
      for target in graph_edges[source]:
        if source not in nodes:
          n[source] = classification
        if target not in nodes:
          n[target] = classification
        edges.add((source, target))
        labeled_edges[(source, target)] = classification
    if len(edges) != 0:
      a.append(edges)
      a.append(n)
      a.append(labeled_edges)
      c.append(a)
      b.append(classification)

USER = {}
USER['data'] = c
USER['target'] = b

In [ ]:
from platform import python_version

print(python_version())

3.7.15


In [ ]:
G = USER['data']
y = USER['target']
G = np.asarray(G)
y = np.asarray(y)
print(G.ndim)
print(y.ndim)
print(type(G[1][1]))
print(len(G))
print(G[0])
print(len(y))

2
1
<class 'dict'>
988
[{(0, 86), (0, 378), (0, 175), (0, 532), (0, 499), (0, 76), (0, 197), (0, 522), (0, 489), (0, 643), (0, 98), (0, 251), (0, 544), (0, 271), (0, 697), (0, 408), (0, 17), (0, 582), (0, 293), (0, 735), (0, 446), (0, 55), (0, 636), (0, 347), (0, 128), (0, 757), (0, 468), (0, 45), (0, 369), (0, 166), (0, 747), (0, 458), (0, 67), (0, 220), (0, 513), (0, 480), (0, 666), (0, 121), (0, 242), (0, 551), (0, 262), (0, 688), (0, 415), (0, 232), (0, 605), (0, 316), (0, 726), (0, 437), (0, 14), (0, 627), (0, 338), (0, 135), (0, 716), (0, 427), (0, 36), (0, 617), (0, 328), (0, 189), (0, 738), (0, 449), (0, 90), (0, 366), (0, 211), (0, 536), (0, 487), (0, 657), (0, 112), (0, 201), (0, 574), (0, 285), (0, 695), (0, 406), (0, 239), (0, 596), (0, 307), (0, 685), (0, 396), (0, 5), (0, 586), (0, 297), (0, 158), (0, 707), (0, 418), (0, 59), (0, 608), (0, 335), (0, 180), (0, 761), (0, 472), (0, 81), (0, 357), (0, 170), (0, 543), (0, 510), (0, 119), (0, 192), (0, 565), (0, 276), (0, 654),

In [ ]:
# Initialize the WL subtree kernel framework

!pip install grakel
from grakel.kernels import WeisfeilerLehman, VertexHistogram
wl_kernel = WeisfeilerLehman(n_iter=10, normalize=False, base_graph_kernel=VertexHistogram)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedKFold

X=G
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

K_train = wl_kernel.fit_transform(X_train)
K_test = wl_kernel.transform(X_test)

In [ ]:
models = []
models.append(("AdaBoost",AdaBoostClassifier()))
models.append(("Multi Layer Perceptron",MLPClassifier()))
models.append(("K Nearest Neighbors",KNeighborsClassifier()))
models.append(("Gaussian Naive Bayes",GaussianNB()))
models.append(("Decision Tree",DecisionTreeClassifier()))
models.append(("Random Forest",RandomForestClassifier()))
models.append(("Support Vector Classification",SVC()))
models.append(("Logistic Regression",LogisticRegression()))


for i in range(2,11):
  for n in range(2, i):
    print('Results for ', n ,' fold cross-validation')
    for name,model in models:
      kfold = KFold(n_splits=n, random_state=42, shuffle=True)
    
      model.fit(K_train, y_train)
      cv_result = cross_val_score(model,K_train,y_train, cv = kfold, scoring = "accuracy")
      print("Accuracy of ", name," with Cross Validation is: {:.2f} %".format(cv_result.mean()*100))
  print()


Results for  2  fold cross-validation
Accuracy of  AdaBoost  with Cross Validation is: 100.00 %
Accuracy of  Multi Layer Perceptron  with Cross Validation is: 100.00 %
Accuracy of  K Nearest Neighbors  with Cross Validation is: 99.24 %
Accuracy of  Gaussian Naive Bayes  with Cross Validation is: 98.73 %
Accuracy of  Decision Tree  with Cross Validation is: 100.00 %
Accuracy of  Random Forest  with Cross Validation is: 100.00 %
Accuracy of  Support Vector Classification  with Cross Validation is: 84.56 %
Accuracy of  Logistic Regression  with Cross Validation is: 100.00 %

Results for  2  fold cross-validation
Accuracy of  AdaBoost  with Cross Validation is: 100.00 %
Accuracy of  Multi Layer Perceptron  with Cross Validation is: 100.00 %
Accuracy of  K Nearest Neighbors  with Cross Validation is: 99.24 %
Accuracy of  Gaussian Naive Bayes  with Cross Validation is: 98.73 %
Accuracy of  Decision Tree  with Cross Validation is: 100.00 %
Accuracy of  Random Forest  with Cross Validation is

In [ ]:
# Split dataset into trainig and testing
from sklearn.model_selection import train_test_split
from grakel.kernels import WeisfeilerLehman, VertexHistogram

# Matrices to store train and test dataset
def wl_kernel_fit_transform(G, y):
  G_train, G_test, y_train, y_test = train_test_split(G, y, random_state=42, shuffle=True)
  print(len(G_train))
  print(len(y_train))
  wl_kernel = WeisfeilerLehman(n_iter=10, normalize=False, base_graph_kernel=VertexHistogram)
  K_train = wl_kernel.fit_transform(G_train)
  K_test = wl_kernel.transform(G_test)
  answer = [K_train, K_test, y_train, y_test]
  print(type(answer))
  print(answer)
  return answer

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

k_matrices = wl_kernel_fit_transform(G,y)
clf = SVC(kernel='precomputed')
clf.fit(k_matrices[0], k_matrices[2])

y_prediction = clf.predict(k_matrices[1])

# print the classification accuracy
from sklearn import metrics

accuracies = metrics.accuracy_score(k_matrices[3], y_prediction)

print("ACC: {}".format(accuracy_score(k_matrices[3], y_prediction)))
print("ROC: {}".format(roc_auc_score(k_matrices[3], y_prediction)))
print("F1: {}".format(f1_score(k_matrices[3], y_prediction)))
print("Precision: {}".format(precision_score(k_matrices[3], y_prediction)))
print("Recall: {}\n".format(recall_score(k_matrices[3], y_prediction)))

741
741
<class 'list'>
[array([[    203739,          0,          0, ...,     384866,          0,
                 0],
       [         0,   64585076,  139407692, ...,          0,  795064600,
          14669396],
       [         0,  139407692,  300913319, ...,          0, 1716157120,
          31664082],
       ...,
       [    384866,          0,          0, ...,     727064,          0,
                 0],
       [         0,  795064600, 1716157120, ...,          0, 9787521566,
         180585360],
       [         0,   14669396,   31664082, ...,          0,  180585360,
           3331976]]), array([[          0,  1241771116,  2680378862, ...,           0,
        15286633852,   282047156],
       [          0,  1145523532,  2472627229, ...,           0,
        14101792652,   260186212],
       [  118473569,           0,           0, ...,   223809756,
                  0,           0],
       ...,
       [   23446973,           0,           0, ...,    44293942,
                  0, 

In [ ]:
# # Employ the random forest classifier
from sklearn.ensemble import RandomForestClassifier

k_matrices = wl_kernel_fit_transform(G,y)
clf = RandomForestClassifier(max_depth=5, random_state=42)
clf.fit(k_matrices[0], k_matrices[2])

y_prediction = clf.predict(k_matrices[1])

# print the classification accuracy
from sklearn import metrics

print("ACC: {}".format(accuracy_score(k_matrices[3], y_prediction)))
print("ROC: {}".format(roc_auc_score(k_matrices[3], y_prediction)))
print("F1: {}".format(f1_score(k_matrices[3], y_prediction)))
print("Precision: {}".format(precision_score(k_matrices[3], y_prediction)))
print("Recall: {}\n".format(recall_score(k_matrices[3], y_prediction)))

741
741
<class 'list'>
[array([[    203739,          0,          0, ...,     384866,          0,
                 0],
       [         0,   64585076,  139407692, ...,          0,  795064600,
          14669396],
       [         0,  139407692,  300913319, ...,          0, 1716157120,
          31664082],
       ...,
       [    384866,          0,          0, ...,     727064,          0,
                 0],
       [         0,  795064600, 1716157120, ...,          0, 9787521566,
         180585360],
       [         0,   14669396,   31664082, ...,          0,  180585360,
           3331976]]), array([[          0,  1241771116,  2680378862, ...,           0,
        15286633852,   282047156],
       [          0,  1145523532,  2472627229, ...,           0,
        14101792652,   260186212],
       [  118473569,           0,           0, ...,   223809756,
                  0,           0],
       ...,
       [   23446973,           0,           0, ...,    44293942,
                  0, 

In [ ]:
# # Employ the adaboost classifier
from sklearn.ensemble import AdaBoostClassifier

k_matrices = wl_kernel_fit_transform(G,y)
clf = AdaBoostClassifier(n_estimators=100, random_state=42)
clf.fit(k_matrices[0], k_matrices[2])

y_prediction = clf.predict(k_matrices[1])

# print the classification accuracy
from sklearn import metrics

accuracies = metrics.accuracy_score(k_matrices[3], y_prediction)
print("ACC: {}".format(accuracy_score(k_matrices[3], y_prediction)))
print("ROC: {}".format(roc_auc_score(k_matrices[3], y_prediction)))
print("F1: {}".format(f1_score(k_matrices[3], y_prediction)))
print("Precision: {}".format(precision_score(k_matrices[3], y_prediction)))
print("Recall: {}\n".format(recall_score(k_matrices[3], y_prediction)))

741
741
<class 'list'>
[array([[    203739,          0,          0, ...,     384866,          0,
                 0],
       [         0,   64585076,  139407692, ...,          0,  795064600,
          14669396],
       [         0,  139407692,  300913319, ...,          0, 1716157120,
          31664082],
       ...,
       [    384866,          0,          0, ...,     727064,          0,
                 0],
       [         0,  795064600, 1716157120, ...,          0, 9787521566,
         180585360],
       [         0,   14669396,   31664082, ...,          0,  180585360,
           3331976]]), array([[          0,  1241771116,  2680378862, ...,           0,
        15286633852,   282047156],
       [          0,  1145523532,  2472627229, ...,           0,
        14101792652,   260186212],
       [  118473569,           0,           0, ...,   223809756,
                  0,           0],
       ...,
       [   23446973,           0,           0, ...,    44293942,
                  0, 

In [ ]:
# # Employ the mlp classifier
from sklearn.neural_network import MLPClassifier

k_matrices = wl_kernel_fit_transform(G,y)
clf = MLPClassifier(random_state=42, max_iter=300)
clf.fit(k_matrices[0], k_matrices[2])

y_prediction = clf.predict(k_matrices[1])

# print the classification accuracy
from sklearn import metrics

accuracies = metrics.accuracy_score(k_matrices[3], y_prediction)

print("ACC: {}".format(accuracy_score(k_matrices[3], y_prediction)))
print("ROC: {}".format(roc_auc_score(k_matrices[3], y_prediction)))
print("F1: {}".format(f1_score(k_matrices[3], y_prediction)))
print("Precision: {}".format(precision_score(k_matrices[3], y_prediction)))
print("Recall: {}\n".format(recall_score(k_matrices[3], y_prediction)))

741
741
<class 'list'>
[array([[    203739,          0,          0, ...,     384866,          0,
                 0],
       [         0,   64585076,  139407692, ...,          0,  795064600,
          14669396],
       [         0,  139407692,  300913319, ...,          0, 1716157120,
          31664082],
       ...,
       [    384866,          0,          0, ...,     727064,          0,
                 0],
       [         0,  795064600, 1716157120, ...,          0, 9787521566,
         180585360],
       [         0,   14669396,   31664082, ...,          0,  180585360,
           3331976]]), array([[          0,  1241771116,  2680378862, ...,           0,
        15286633852,   282047156],
       [          0,  1145523532,  2472627229, ...,           0,
        14101792652,   260186212],
       [  118473569,           0,           0, ...,   223809756,
                  0,           0],
       ...,
       [   23446973,           0,           0, ...,    44293942,
                  0, 

In [ ]:
# # Employ the GaussianNB classifier
from sklearn.naive_bayes import GaussianNB

k_matrices = wl_kernel_fit_transform(G,y)
clf = GaussianNB()
clf.fit(k_matrices[0], k_matrices[2])

y_prediction = clf.predict(k_matrices[1])

# print the classification accuracy
from sklearn import metrics

accuracies = metrics.accuracy_score(k_matrices[3], y_prediction)

print("ACC: {}".format(accuracy_score(k_matrices[3], y_prediction)))
print("ROC: {}".format(roc_auc_score(k_matrices[3], y_prediction)))
print("F1: {}".format(f1_score(k_matrices[3], y_prediction)))
print("Precision: {}".format(precision_score(k_matrices[3], y_prediction)))
print("Recall: {}\n".format(recall_score(k_matrices[3], y_prediction)))

741
741
<class 'list'>
[array([[    203739,          0,          0, ...,     384866,          0,
                 0],
       [         0,   64585076,  139407692, ...,          0,  795064600,
          14669396],
       [         0,  139407692,  300913319, ...,          0, 1716157120,
          31664082],
       ...,
       [    384866,          0,          0, ...,     727064,          0,
                 0],
       [         0,  795064600, 1716157120, ...,          0, 9787521566,
         180585360],
       [         0,   14669396,   31664082, ...,          0,  180585360,
           3331976]]), array([[          0,  1241771116,  2680378862, ...,           0,
        15286633852,   282047156],
       [          0,  1145523532,  2472627229, ...,           0,
        14101792652,   260186212],
       [  118473569,           0,           0, ...,   223809756,
                  0,           0],
       ...,
       [   23446973,           0,           0, ...,    44293942,
                  0, 

In [ ]:
# # Employ the knn classifier
from sklearn.neighbors import KNeighborsClassifier

k_matrices = wl_kernel_fit_transform(G,y)
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(k_matrices[0], k_matrices[2])

y_prediction = clf.predict(k_matrices[1])

# print the classification accuracy
from sklearn import metrics

accuracies = metrics.accuracy_score(k_matrices[3], y_prediction)

print("ACC: {}".format(accuracy_score(k_matrices[3], y_prediction)))
print("ROC: {}".format(roc_auc_score(k_matrices[3], y_prediction)))
print("F1: {}".format(f1_score(k_matrices[3], y_prediction)))
print("Precision: {}".format(precision_score(k_matrices[3], y_prediction)))
print("Recall: {}\n".format(recall_score(k_matrices[3], y_prediction)))

741
741
<class 'list'>
[array([[    203739,          0,          0, ...,     384866,          0,
                 0],
       [         0,   64585076,  139407692, ...,          0,  795064600,
          14669396],
       [         0,  139407692,  300913319, ...,          0, 1716157120,
          31664082],
       ...,
       [    384866,          0,          0, ...,     727064,          0,
                 0],
       [         0,  795064600, 1716157120, ...,          0, 9787521566,
         180585360],
       [         0,   14669396,   31664082, ...,          0,  180585360,
           3331976]]), array([[          0,  1241771116,  2680378862, ...,           0,
        15286633852,   282047156],
       [          0,  1145523532,  2472627229, ...,           0,
        14101792652,   260186212],
       [  118473569,           0,           0, ...,   223809756,
                  0,           0],
       ...,
       [   23446973,           0,           0, ...,    44293942,
                  0, 

In [ ]:
# # Employ the DecisionTreeClassifier classifier
from sklearn.tree import DecisionTreeClassifier

k_matrices = wl_kernel_fit_transform(G,y)
clf = DecisionTreeClassifier(random_state=42)
clf.fit(k_matrices[0], k_matrices[2])

y_prediction = clf.predict(k_matrices[1])

# print the classification accuracy
from sklearn import metrics

accuracies = metrics.accuracy_score(k_matrices[3], y_prediction)

print("ACC: {}".format(accuracy_score(k_matrices[3], y_prediction)))
print("ROC: {}".format(roc_auc_score(k_matrices[3], y_prediction)))
print("F1: {}".format(f1_score(k_matrices[3], y_prediction)))
print("Precision: {}".format(precision_score(k_matrices[3], y_prediction)))
print("Recall: {}\n".format(recall_score(k_matrices[3], y_prediction)))

741
741
<class 'list'>
[array([[    203739,          0,          0, ...,     384866,          0,
                 0],
       [         0,   64585076,  139407692, ...,          0,  795064600,
          14669396],
       [         0,  139407692,  300913319, ...,          0, 1716157120,
          31664082],
       ...,
       [    384866,          0,          0, ...,     727064,          0,
                 0],
       [         0,  795064600, 1716157120, ...,          0, 9787521566,
         180585360],
       [         0,   14669396,   31664082, ...,          0,  180585360,
           3331976]]), array([[          0,  1241771116,  2680378862, ...,           0,
        15286633852,   282047156],
       [          0,  1145523532,  2472627229, ...,           0,
        14101792652,   260186212],
       [  118473569,           0,           0, ...,   223809756,
                  0,           0],
       ...,
       [   23446973,           0,           0, ...,    44293942,
                  0, 

In [ ]:
# # Employ the LogisticRegression classifier
from sklearn.linear_model import LogisticRegression

k_matrices = wl_kernel_fit_transform(G,y)
clf = LogisticRegression(random_state=42)
clf.fit(k_matrices[0], k_matrices[2])

y_prediction = clf.predict(k_matrices[1])

# print the classification accuracy
from sklearn import metrics

accuracies = metrics.accuracy_score(k_matrices[3], y_prediction)

print("ACC: {}".format(accuracy_score(k_matrices[3], y_prediction)))
print("ROC: {}".format(roc_auc_score(k_matrices[3], y_prediction)))
print("F1: {}".format(f1_score(k_matrices[3], y_prediction)))
print("Precision: {}".format(precision_score(k_matrices[3], y_prediction)))
print("Recall: {}\n".format(recall_score(k_matrices[3], y_prediction)))

741
741
<class 'list'>
[array([[    203739,          0,          0, ...,     384866,          0,
                 0],
       [         0,   64585076,  139407692, ...,          0,  795064600,
          14669396],
       [         0,  139407692,  300913319, ...,          0, 1716157120,
          31664082],
       ...,
       [    384866,          0,          0, ...,     727064,          0,
                 0],
       [         0,  795064600, 1716157120, ...,          0, 9787521566,
         180585360],
       [         0,   14669396,   31664082, ...,          0,  180585360,
           3331976]]), array([[          0,  1241771116,  2680378862, ...,           0,
        15286633852,   282047156],
       [          0,  1145523532,  2472627229, ...,           0,
        14101792652,   260186212],
       [  118473569,           0,           0, ...,   223809756,
                  0,           0],
       ...,
       [   23446973,           0,           0, ...,    44293942,
                  0, 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

k_matrices = wl_kernel_fit_transform(G,y)
clf = DecisionTreeClassifier(random_state=42)
clf.fit(k_matrices[0], k_matrices[2])

y_prediction = clf.predict(k_matrices[1])

# print the classification accuracy
from sklearn import metrics

accuracies = metrics.accuracy_score(k_matrices[3], y_prediction)

print("ACC: {}".format(accuracy_score(k_matrices[3], y_prediction)))
print("ROC: {}".format(roc_auc_score(k_matrices[3], y_prediction)))
print("F1: {}".format(f1_score(k_matrices[3], y_prediction)))
print("Precision: {}".format(precision_score(k_matrices[3], y_prediction)))
print("Recall: {}\n".format(recall_score(k_matrices[3], y_prediction)))

741
741
<class 'list'>
[array([[    203739,          0,          0, ...,     384866,          0,
                 0],
       [         0,   64585076,  139407692, ...,          0,  795064600,
          14669396],
       [         0,  139407692,  300913319, ...,          0, 1716157120,
          31664082],
       ...,
       [    384866,          0,          0, ...,     727064,          0,
                 0],
       [         0,  795064600, 1716157120, ...,          0, 9787521566,
         180585360],
       [         0,   14669396,   31664082, ...,          0,  180585360,
           3331976]]), array([[          0,  1241771116,  2680378862, ...,           0,
        15286633852,   282047156],
       [          0,  1145523532,  2472627229, ...,           0,
        14101792652,   260186212],
       [  118473569,           0,           0, ...,   223809756,
                  0,           0],
       ...,
       [   23446973,           0,           0, ...,    44293942,
                  0, 

In [ ]:
k_matrices = wl_kernel_fit_transform(G,y)
clf = SVC(random_state=42)
clf.fit(k_matrices[0], k_matrices[2])

y_prediction = clf.predict(k_matrices[1])

# print the classification accuracy
from sklearn import metrics

accuracies = metrics.accuracy_score(k_matrices[3], y_prediction)

print("ACC: {}".format(accuracy_score(k_matrices[3], y_prediction)))
print("ROC: {}".format(roc_auc_score(k_matrices[3], y_prediction)))
print("F1: {}".format(f1_score(k_matrices[3], y_prediction)))
print("Precision: {}".format(precision_score(k_matrices[3], y_prediction)))
print("Recall: {}\n".format(recall_score(k_matrices[3], y_prediction)))

741
741
<class 'list'>
[array([[    203739,          0,          0, ...,     384866,          0,
                 0],
       [         0,   64585076,  139407692, ...,          0,  795064600,
          14669396],
       [         0,  139407692,  300913319, ...,          0, 1716157120,
          31664082],
       ...,
       [    384866,          0,          0, ...,     727064,          0,
                 0],
       [         0,  795064600, 1716157120, ...,          0, 9787521566,
         180585360],
       [         0,   14669396,   31664082, ...,          0,  180585360,
           3331976]]), array([[          0,  1241771116,  2680378862, ...,           0,
        15286633852,   282047156],
       [          0,  1145523532,  2472627229, ...,           0,
        14101792652,   260186212],
       [  118473569,           0,           0, ...,   223809756,
                  0,           0],
       ...,
       [   23446973,           0,           0, ...,    44293942,
                  0, 